**CAUTION**

The parsing is not perfect. See 'Conjure Woodland Beings' (4th level) as an example. It contains a list, which is not parsed.

There's also spells with tables, these are not parsed either. Nevertheless, the html is here so you can tweak the parsing code to retrieve this missing information.

There might also be errors in some entries due to inconsistent formatting.

In [1]:
from glob import glob
from lxml import html

import re
import pandas as pd

In [2]:
def get_all_text(elem):
    return elem.xpath('string(.)')

regex_special = re.compile(r' \((.*?)\)')
regex_summary = re.compile(r'(.*?): ((?:\n|.)*?)\n')
regex_higher = re.compile(r'At Higher Levels(?:\:|\.) (.*?)', flags=re.IGNORECASE)
regex_components = re.compile(r'(?:(V|S|M).*?)*?(?: \((.+?)\))?')

In [3]:
data = []

for path in glob('raw/*.txt'):
    with open(path, encoding='unicode_escape') as f:
        tree = html.fromstring(f.read())
        
    name, = tree.xpath('//div[starts-with(@class, "page-title")]/span')
    name = name.text

    ps = tree.xpath('//div[@id="page-content"]//p')
    
    source, level_school, summary = ps[:3]
    spell_lists = ps[-1]

    source = get_all_text(source).replace('Source: ', '')
    level_school = get_all_text(level_school)
    if ' cantrip' in level_school:
        level_school = level_school.replace(' cantrip', '')
        level_school = 'Cantrip ' + level_school
        
    # Use regex to extract any special type
    special = regex_special.findall(level_school)
    for x in special:
        level_school = level_school.replace(f' ({x})', '')

    level, school = level_school.split(' ')
    level = 0 if level.lower() == 'cantrip' else int(level[0])
    school = school.title() # capitalize first word
    summary = get_all_text(summary) + '\n' # also add a \n for the regex
    summary = dict(
        (k, v.replace('\n', ' '))
        for (k, v) in regex_summary.findall(summary)
    )
    concentration = 'Concentration' in summary.get('Duration', '')
    
    components = summary.pop('Components')
    components = list(regex_components.fullmatch(components).groups())
    if 'M' in components:
        material = components[-1]
        components = components[:-1]
    else:
        material = ''

    description = [ p.xpath('string(.)') for p in ps[3:-1] ]
    for i, line in enumerate(description):
        match = regex_higher.fullmatch(line)
        if match:
            at_higher_levels = '\n'.join([match.groups()[0]] + description[i+1:]).strip()
            description = description[:i]
            break
    else:
        at_higher_levels = ''
    
    description = '\n'.join(description)
    
    spell_lists = spell_lists.xpath('.//a/text()')
    
    d = dict(
        name=name,
        source=source,
        level=level,
        school=school,
        components=components,
        material=material,
        concentration=concentration,
        special=special,
        description=description,
        at_higher_levels=at_higher_levels,
        spell_lists=spell_lists
    )
    
    for k, v in summary.items():
        d[k] = v
    
    data.append(d)

In [4]:
df = pd.DataFrame(data)

df

,name,source,level,school,components,material,concentration,special,description,at_higher_levels,spell_lists,Casting Time,Range,Duration
0,Abi-Dalzim's Horrid Wilting,Xanathar's Guide to Everything,8,Necromancy,[M],a bit of sponge,False,[],You draw the moisture from every creature in a...,,"[Sorcerer, Wizard]",1 action,150 feet,Instantaneous
1,Absorb Elements,Xanathar's Guide to Everything,1,Abjuration,"[S, None]",,False,[],The spell captures some of the incoming energy...,When you cast this spell using a spell slot of...,"[Artificer, Druid, Ranger, Sorcerer, Wizard]","1 reaction, which you take when you take acid,...",Self,1 round
2,Acid Splash,Player's Handbook,0,Conjuration,"[S, None]",,False,[],You hurl a bubble of acid. Choose one creature...,This spell’s damage increases by 1d6 when you ...,"[Artificer, Sorcerer, Wizard]",1 action,60 feet,Instantaneous
3,Acid Stream (UA),Unearthed Arcana 70 - Spells and Magic Tattoos,1,Evocation,[M],a bit of rotten food,True,[],A stream of acid emanates from you in a line 3...,When you cast this spell using a spell slot of...,"[Sorcerer, Wizard]",1 action,Self (30-foot line),"Concentration, up to 1 minute"
4,Aganazzar's Scorcher,Xanathar's Guide to Everything,2,Evocation,[M],a red dragon’s scale,False,[],A line of roaring flame 30 feet long and 5 fee...,When you cast this spell using a spell slot of...,"[Sorcerer, Wizard]",1 action,30 feet,Instantaneous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,Wrath Of Nature,Xanathar's Guide to Everything,5,Evocation,"[S, None]",,True,[],You call out to the spirits of nature to rouse...,,"[Druid, Ranger]",1 action,120 feet,"Concentration, up to 1 minute"
569,Wrathful Smite,Player's Handbook,1,Evocation,"[V, None]",,True,[],The next time you hit with a melee weapon atta...,,[Paladin],1 bonus action,Self,"Concentration, up to 1 minute"
570,Wristpocket,Explorer's Guide to Wildemount,2,Conjuration,"[S, None]",,True,"[ritual, dunamancy]","You flick your wrist, causing one object in yo...",,[Wizard (Dunamancy)],1 action,Self,"Concentration, up to 1 hour"
571,Zephyr Strike,Xanathar's Guide to Everything,1,Transmutation,"[V, None]",,True,[],"You move like the wind. For the duration, your...",,[Ranger],1 bonus action,Self,"Concentration, up to 1 minute"


In [5]:
df.to_csv('parsed.csv', index=False)
df.to_excel('parsed.xlsx', index=False)